### 모듈 설치

In [4]:
%pip install psycopg2-binary dotenv

Note: you may need to restart the kernel to use updated packages.


#### postgresql 연결

In [6]:
import os
from dotenv import load_dotenv
from pathlib import Path

# 환경 변수 로드
load_dotenv()

Postgre_HOST = os.getenv("Postgre_HOST")
Postgre_PORT = os.getenv("Postgre_PORT")
Postgre_DB = os.getenv("Postgre_DB")
Postgre_USER = os.getenv("Postgre_USER")
Postgre_PASSWORD = os.getenv("Postgre_PASSWORD")

print(Postgre_HOST)

localhost


#### Postgresql 에 boxoffice 테이블 만들고 raw_data 밀어넣기

In [11]:
import psycopg2

try:
    conn = psycopg2.connect(host=Postgre_HOST, dbname=Postgre_DB, user=Postgre_USER, password=Postgre_PASSWORD)
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS boxoffice")

    sql = '''
    CREATE TABLE boxoffice (
        movieCd     BIGINT,         -- 영화 코드
        showRange   DATE,           -- 상영 날짜
        rank        INT,            -- 순위
        salesAmt    BIGINT,         -- 매출액
        salesShare  DOUBLE PRECISION, -- 매출 점유율 (퍼센트, 소수점 가능)
        salesInten  BIGINT,         -- 매출 증감
        salesChange DOUBLE PRECISION, -- 매출 증감률
        salesAcc    BIGINT,         -- 누적 매출
        audiCnt     BIGINT,         -- 관객 수
        audiInten   BIGINT,         -- 관객 증감
        audiChange  DOUBLE PRECISION, -- 관객 증감률
        audiAcc     BIGINT,         -- 누적 관객
        scrnCnt     INT,            -- 상영 스크린 수
        showCnt     INT             -- 상영 횟수
    );
    '''

    cursor.execute(sql)
    print('table create success')
    conn.commit()

except Exception as e:
    print("fail:", e)

finally:
    conn.close()

table create success


In [12]:
from pathlib import Path
import json
import glob

PROJECT_DIR = Path.cwd().parent
json_dir = PROJECT_DIR / 'raw_data/boxoffice'
json_files = glob.glob(os.path.join(json_dir, 'dailyBoxOffice_*.json'))

try:
    conn = psycopg2.connect(
        host=Postgre_HOST,
        dbname=Postgre_DB,
        user=Postgre_USER,
        password=Postgre_PASSWORD
    )
    cursor = conn.cursor()

    insert_sql = '''
    INSERT INTO boxoffice (
        movieCd, showRange, rank, salesAmt, salesShare, salesInten,
        salesChange, salesAcc, audiCnt, audiInten, audiChange,
        audiAcc, scrnCnt, showCnt
    ) VALUES (
        %(movieCd)s, %(showRange)s, %(rank)s, %(salesAmt)s, %(salesShare)s, %(salesInten)s,
        %(salesChange)s, %(salesAcc)s, %(audiCnt)s, %(audiInten)s, %(audiChange)s,
        %(audiAcc)s, %(scrnCnt)s, %(showCnt)s
    )
    '''

    for file in json_files:
        print(f"Processing {file} ...")
        with open(file, 'r', encoding='utf-8') as f:
            for line in f:
                data = json.loads(line.strip())
                cursor.execute(insert_sql, data)

    conn.commit()
    print("All JSON files inserted successfully.")

except Exception as e:
    print("Error:", e)

finally:
    conn.close()

Processing /Users/kang/4Ward/BI-Dashboard/raw_data/boxoffice/dailyBoxOffice_20250927.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/boxoffice/dailyBoxOffice_20250911.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/boxoffice/dailyBoxOffice_20250907.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/boxoffice/dailyBoxOffice_20250906.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/boxoffice/dailyBoxOffice_20250910.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/boxoffice/dailyBoxOffice_20250930.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/boxoffice/dailyBoxOffice_20250926.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/boxoffice/dailyBoxOffice_20250917.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/boxoffice/dailyBoxOffice_20250901.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/boxoffice/dailyBoxOffice_20250921.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/boxoffice

#### Postgresql 에 movie_info 테이블 만들고 raw_data 밀어넣기

In [13]:
import psycopg2

try:
    conn = psycopg2.connect(host=Postgre_HOST, dbname=Postgre_DB, user=Postgre_USER, password=Postgre_PASSWORD)
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS movie_info")

    sql = '''
    CREATE TABLE movie_info (
        movieCd     BIGINT,       -- 영화 코드, 숫자가 클 수 있음
        movieNm     TEXT,         -- 영화 이름
        openDt      DATE,         -- 개봉일, 'YYYY-MM-DD' 형식 문자열 사용 가능
        genre       TEXT          -- 장르
    );
    '''

    cursor.execute(sql)
    print('table create success')
    conn.commit()

except Exception as e:
    print("fail:", e)

finally:
    conn.close()

table create success


In [14]:
from pathlib import Path
import json
import glob

PROJECT_DIR = Path.cwd().parent
json_dir = PROJECT_DIR / 'raw_data/movieInfo'
json_files = glob.glob(os.path.join(json_dir, 'movieInfo_*.json'))

try:
    conn = psycopg2.connect(
        host=Postgre_HOST,
        dbname=Postgre_DB,
        user=Postgre_USER,
        password=Postgre_PASSWORD
    )
    cursor = conn.cursor()

    insert_sql = '''
    INSERT INTO movie_info (
        movieCd, movieNm, openDt, genre
    ) VALUES (
        %(movieCd)s, %(movieNm)s, %(openDt)s, %(genre)s
    )
    '''

    for file in json_files:
        print(f"Processing {file} ...")
        with open(file, 'r', encoding='utf-8') as f:
            for line in f:
                data = json.loads(line.strip())
                cursor.execute(insert_sql, data)

    conn.commit()
    print("All JSON files inserted successfully.")

except Exception as e:
    print("Error:", e)

finally:
    conn.close()

Processing /Users/kang/4Ward/BI-Dashboard/raw_data/movieInfo/movieInfo_20210546.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/movieInfo/movieInfo_20256202.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/movieInfo/movieInfo_20256701.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/movieInfo/movieInfo_20256185.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/movieInfo/movieInfo_20240206.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/movieInfo/movieInfo_20256281.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/movieInfo/movieInfo_20256757.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/movieInfo/movieInfo_20256741.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/movieInfo/movieInfo_20256864.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/movieInfo/movieInfo_20256034.json ...
Processing /Users/kang/4Ward/BI-Dashboard/raw_data/movieInfo/movieInfo_20257229.json ...
Processing /Users/kan